# 2장. 싱글톤 디자인 패턴

## 목적

- 클래스에 대한 단일 객체 생성
- 전역 객체 제공
- 공유된 리소스에 대한 동시 접근 제어

## 주의점

- 전역 변수의 값이 변경된 것을 모르고 애플리케이션에서 사용될 수 있음
- 같은 객체에 대한 여러 참조자가 생김
- 전역 변수에 종속적인 모든 클래스 간 상호관계가 복잡해짐

In [59]:
# 파이썬 싱글톤 패턴
class Singleton(object):
    def __new__(cls):
        if not hasattr(cls, "_instance"):
            cls.instance = super(Singleton, cls).__new__(cls)
        return cls.instance


s = Singleton()
print("Object created", s)

s1 = Singleton()
print("Object created", s1)

# $ py singeton.py
# Object created <__main__.Singleton object at 0x102fad9d0>
# Object created <__main__.Singleton object at 0x102fada10>

Object created <__main__.Singleton object at 0x118230850>
Object created <__main__.Singleton object at 0x107d0e550>


In [2]:
# 게으른 초기화
class Singleton:
    __instance = None

    def __init__(self):
        if not Singleton.__instance:
            print("__init__ method called.")
        else:
            print("Instance already created:", self.getInstance())

    @classmethod
    def getInstance(cls):
        if not cls.__instance:
            cls.__instance = Singleton()
        return cls.__instance


s = Singleton()  ## 클래스 초기화, 객체 생성하지 않음
print("Object created", Singleton.getInstance())  # 객체 생성
s1 = Singleton()  ## 객체는 이미 생성됐음

# __init__ method called.
# __init__ method called.
# Object created <__main__.Singleton object at 0x10425ff10>
# Instance already created: <__main__.Singleton object at 0x10425ff10>

__init__ method called.
__init__ method called.
Object created <__main__.Singleton object at 0x1043c1f10>
Instance already created: <__main__.Singleton object at 0x1043c1f10>


In [1]:
# 모노스테이트 싱글톤 패턴
class Borg:
    __shared_state = {"1": "2"}

    def __init__(self):
        self.x = 1
        self.__dict__ = self.__shared_state
        pass


b = Borg()
b1 = Borg()
b.x = 4

print(f"Borg Object 'b': {b}")
print(f"Borg Object 'b1': {b1}")
print(f"Object State 'b': {b.__dict__}")
print(f"Object State 'b1': {b1.__dict__}")

# Borg Object 'b': <__main__.Borg object at 0x107cb7490>
# Borg Object 'b1': <__main__.Borg object at 0x107ca81d0>
# Object State 'b': {'1': '2', 'x': 4}
# Object State 'b1': {'1': '2', 'x': 4}

Borg Object 'b': <__main__.Borg object at 0x107cb7490>
Borg Object 'b1': <__main__.Borg object at 0x107ca81d0>
Object State 'b': {'1': '2', 'x': 4}
Object State 'b1': {'1': '2', 'x': 4}


In [3]:
# __new__ 메서드를 사용해 구현
class Borg(object):
    _shared_state = {}

    def __new__(cls, *args, **kwargs):
        obj = super(Borg, cls).__new__(cls, *args, **kwargs)
        obj.__dict__ = cls._shared_state
        return obj


b = Borg()
b1 = Borg()
b.x = 4

print(f"Borg Object 'b': {b}")
print(f"Borg Object 'b1': {b1}")
print(f"Object State 'b': {b.__dict__}")
print(f"Object State 'b1': {b1.__dict__}")

# Borg Object 'b': <__main__.Borg object at 0x107ccf450>
# Borg Object 'b1': <__main__.Borg object at 0x107cb7490>
# Object State 'b': {'x': 4}
# Object State 'b1': {'x': 4}

Borg Object 'b': <__main__.Borg object at 0x107ccf450>
Borg Object 'b1': <__main__.Borg object at 0x107cb7490>
Object State 'b': {'x': 4}
Object State 'b1': {'x': 4}


In [7]:
# 메타 클래스
class MyInt(type):
    def __call__(cls, *args, **kwargs):
        print("***** Here's My int *****", args)
        print("Now do whatever you want with these objects...")
        return type.__call__(cls, *args, **kwargs)

class int(metaclass=MyInt):
    def __init__(self, x, y):
        self.x = x
        self.y = y

i = int(4,5)
print(i.__dict__)

# ***** Here's My int ***** (4, 5)
# Now do whatever you want with these objects...
# {'x': 4, 'y': 5}

***** Here's My int ***** (4, 5)
Now do whatever you want with these objects...
{'x': 4, 'y': 5}


In [9]:
# 메타클래스로 싱글톤 패턴 구현
class MetaSingleton(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(MetaSingleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls]


class Logger(metaclass=MetaSingleton):
    pass


logger1 = Logger()
logger2 = Logger()
print(logger1, logger2)

# <__main__.Logger object at 0x1181efc90> <__main__.Logger object at 0x1181efc90>

<__main__.Logger object at 0x1181efc90> <__main__.Logger object at 0x1181efc90>


In [14]:
# 싱글톤 패턴 사용 사례 1
import sqlite3


class MetaSingleton(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(MetaSingleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls]


class Database(metaclass=MetaSingleton):
    connection = None

    def connect(self):
        if self.connection is None:
            self.connection = sqlite3.connect("db.sqlite3")
            self.cursorobj = self.connection.cursor()
        return self.cursorobj

db1 = Database().connect()
db2 = Database().connect()

print("Database Objects DB1", db1)
print("Database Objects DB2", db2)

# Database Objects DB1 <sqlite3.Cursor object at 0x107cee3c0>
# Database Objects DB2 <sqlite3.Cursor object at 0x107cee3c0>

Database Objects DB1 <sqlite3.Cursor object at 0x107cee3c0>
Database Objects DB2 <sqlite3.Cursor object at 0x107cee3c0>


In [60]:
# 싱글톤 패턴 사용 사례 2
class HealthCheck:
    _instance = None
    def __new__(cls, *args, **kwargs):
        if not HealthCheck._instance:
            HealthCheck._instance = super(HealthCheck, \
                cls).__new__(cls, *args, **kwargs)
        return HealthCheck._instance
    def __init__(self):
        self._servers = []
    def addServer(self):
        self._servers.append("Server 1")
        self._servers.append("Server 2")
        self._servers.append("Server 3")
        self._servers.append("Server 4")
    def changeServer(self):
        self._servers.pop()
        self._servers.append("Server 5")


hc1 = HealthCheck()
hc2 = HealthCheck()

hc1.addServer()
print("Schedule health check for servers (1)..")
for i in range(4):
    print("Checking ", hc1._servers[i])

hc2.changeServer()
print("Schedule health check for servers (2)..")
for i in range(4):
    print("Checking ", hc2._servers[i])

# Schedule health check for servers (1)..
# Checking  Server 1
# Checking  Server 2
# Checking  Server 3
# Checking  Server 4
# Schedule health check for servers (2)..
# Checking  Server 1
# Checking  Server 2
# Checking  Server 3
# Checking  Server 5


Schedule health check for servers (1)..
Checking  Server 1
Checking  Server 2
Checking  Server 3
Checking  Server 4
Schedule health check for servers (2)..
Checking  Server 1
Checking  Server 2
Checking  Server 3
Checking  Server 5
